# Machine Translation Project
A deep neural network that functions as part of an end-to-end machine translation pipeline


In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [2]:
import collections 

import helper
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import LSTM, GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Dropout
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam 
from keras.losses import sparse_categorical_crossentropy

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


## Dataset 
This dataset will be used to train and evaluate the pipeline. This dataset contains a small vocabulary. This dataset will allow the model to be trained in a reasonable amount of time. 
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [3]:
# Loading english data
english_sentences = helper.load_data('data/small_vocab_en')
# Loading french data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')
print(type(english_sentences))


Dataset Loaded
<class 'list'>
